# Installing and Imports libraries

In [ ]:
#%pip install pylab

In [1]:
# Imports
import random
import numpy as np
import pandas as pd
from itertools import combinations
from scipy.stats import ttest_ind
import os
import sys
import PIL
import cv2
import re


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# plt
import matplotlib.pyplot as plt
import seaborn as sns
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

In [2]:
print('Python         :', sys.version.split('\n')[0])
print('Numpy          :', np.__version__)

Python         : 3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]
Numpy          : 1.24.2


In [9]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [10]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [11]:
#!pip freeze > requirements.txt

# DATA

In [12]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [13]:
train.columns

In [14]:
print(f'num train cols: {len(train.columns)},  ', f'num test cols: {len(test.columns)}')
print(f'col not in test: {set(train.columns) - set(test.columns)}')

In [15]:
train.info()

In [16]:
train.nunique()

Типы признаков:

* bodyType - категориальный
* brand - категориальный
* color - категориальный
* description - текстовый
* engineDisplacement - числовой, представленный как текст
* enginePower - числовой, представленный как текст
* fuelType - категориальный
* mileage - числовой
* modelDate - числовой
* model_info - категориальный
* name - категориальный, желательно сократить размерность
* numberOfDoors - категориальный
* price - числовой, целевой
* productionDate - числовой
* sell_id - изображение (файл доступен по адресу, основанному на sell_id)
* vehicleConfiguration - не используется (комбинация других столбцов)
* vehicleTransmission - категориальный
* Владельцы - категориальный
* Владение - числовой, представленный как текст
* ПТС - категориальный
* Привод - категориальный
* Руль - категориальный

# EDA

### NaN analisys

In [17]:
df_train = train.copy()
df_test = test.copy()

In [18]:
# NaN analisys
df_train.isna().sum()

In [19]:
# NaN analisys
df_test.isna().sum()

In [20]:
# Функция преобразования 'Владение' в числовой признак
def prepare_vladenie(string, col='Владение', pattern_1 = '\d+'):
    if type(string) == float:
        # Если NaN меняем на 0
        num_mounth = 0
    elif len(string.split('и'))==2:
        nums = re.findall(pattern_1, string)
        num_mounth = int(nums[1]) + int(nums[0])*12
    elif 'мес'in string:
        num_mounth = int(re.findall(pattern_1, string)[0])
    else: 
        num_mounth = int(re.findall(pattern_1, string)[0])*12
    return num_mounth

df_train['Владение'] = df_train.Владение.apply(prepare_vladenie)
df_test['Владение'] = df_test.Владение.apply(prepare_vladenie)

**train**:
1. в **владение** слишком много пропусков, столбец будет использован на создания нового признака: Вар.1: учесть есть/нет NaN и удалить; Вар.2: преобразование в число + nan = 1 мес
2. train.loc[train['Владельцы'].isna(), 'Владельцы'] = '3 или более' т.к. авто 2001 года, но можно и удалить

**test**:
1. в **владение** слишком много пропусков, столбец будет использован на создания нового признака: Вар.1: учесть есть/нет NaN и удалить; Вар.2: преобразование в число + nan = 1 мес

In [21]:
print(len(df_train))
df_train.dropna(subset=['Владельцы'], inplace=True)
print(len(df_train))

### Object columns data preanalisys

In [22]:
df_train.dtypes[train.dtypes==object].index

In [23]:
# анализ, где мы можем ошибиться на тестовых данных при обучении
def col_preanalisys(col):
    if len(df_train[col].unique())>=len(df_test[col].unique()):
        print(f'Число уникальных значений в колонке {col} в train больше или равно, чем в test.\n', 
              f'Значения {col} в test, которых нет в train: {set(df_test[col].unique()) - set(df_train[col].unique())}')
        print()
    else:
        print(f'Число уникальных значений в колонке {col} в test больше, чем в train.\n',
              f'Значения {col} в test, которых нет в train: {set(df_test[col].unique()) - set(df_train[col].unique())}')
        print()

In [24]:
for col in set(df_train.dtypes[df_train.dtypes==object].index) - set(['description']):
    col_preanalisys(col)

In [25]:
# функция преобразования колонки name
def ch_name_col(d_frame_name):
    if '4WD' in d_frame_name:
        full_drive = ' 4WD'
    else:
        full_drive = ''
    pattern_1 = ' \d\.\d'
    pattern_2 = '\d\.\d'
    pattern_3 = ' AT'
    if len(re.findall(pattern_1, d_frame_name))!=0:
        return re.split(pattern_1, d_frame_name)[0]+full_drive
    elif len(re.findall(pattern_2, d_frame_name))!=0:
        return 'no_val'+full_drive
    else:
        return re.split(pattern_2, d_frame_name)[0]+full_drive
    
df_train['ch_name'] = df_train.name.apply(ch_name_col)
df_test['ch_name'] = df_test.name.apply(ch_name_col)

In [26]:
df_train.drop(['name'], axis=1, inplace=True)
df_test.drop(['name'], axis=1, inplace=True)

In [27]:
df_train.engineDisplacement.replace('undefined LTR', '0.0 LTR', inplace=True)
df_test.engineDisplacement.replace('undefined LTR', '0.0 LTR', inplace=True)
def ch_engineDisplacement(d_frame_engine_d):
    return float(d_frame_engine_d[:-4])
df_train['ch_engineDisplacement'] = df_train.engineDisplacement.apply(ch_engineDisplacement) 
df_test['ch_engineDisplacement'] = df_test.engineDisplacement.apply(ch_engineDisplacement) 

In [28]:
def ch_enginePower(d_frame_engine_p):
    return int(d_frame_engine_p[:-4])
df_train['ch_enginePower'] = df_train.enginePower.apply(ch_enginePower)
df_test['ch_enginePower'] = df_test.enginePower.apply(ch_enginePower)

### EDA numb_cols

In [29]:
# Функция для вывода статистических характеристик столбцов с числами
def numb_type_analisys(df, col_name, bins_step=1):

    print('\n', '\033[1m' + 'Столбец: ' + col_name, '\033[0m', '\n')

    #print("Количество уникальных значений в столбце {}:".format(col_name))
    #print(dict(df[col_name].value_counts()), '\n')

    #print('Доли уникальных значений в столбце {}:'.format(col_name))
    #print(dict(round(df[col_name].value_counts(normalize=True), 3)), '\n')

    # Данные для анализа выбросов
    IQR = df[col_name].quantile(0.75) - df[col_name].quantile(0.25)
    perc25 = df[col_name].quantile(0.25)
    perc75 = df[col_name].quantile(0.75)
    out_left = perc25 - 1.5*IQR
    out_right = perc75 + 1.5*IQR

    print('Статистические параметры столбца {}:'.format(col_name))
    print(
        '25-й перцентиль: {},'.format(perc25),
        '75-й перцентиль: {},'.format(perc75),
        "IQR: {}, ".format(IQR),
        "Границы выбросов: [{f}, {l}].".format(f=out_left, l=out_right),
        '\n')

    if out_left > min(df[col_name]):
        print('Имеются выбросы в области минимальных значений')
    if out_right < max(df[col_name]):
        print('Имеются выбросы в области максимальных значений')
    if (out_left < min(df[col_name])) and (out_right > max(df[col_name])):
        print('Выбросов нет')

    display(df[col_name].describe())

    df[col_name].hist(bins=np.arange(min(df[col_name]), max(df[col_name])+1, bins_step),
                      align='left',
                      label=col_name)
    plt.legend()

In [30]:
df_train.dtypes[df_train.dtypes!=object].index

### **mileage**

In [31]:
## mileage train
numb_type_analisys(df_train, 'mileage', 
                   bins_step=(df_train.mileage.max() - df_train.mileage.min())//100)

Имеется выброс в области максимальных значений: для автомобиля 2008 года указан пробег 999999 км. Что довольно много. При анализе 'description' указано: "Оригинальный пробег 98500 км", что подозрительно мало, но явно не 999999 км. Отсюда можно данное значение либо удалить, либо исправить на 99999. Попробуем второе.

In [32]:
df_train[df_train['mileage']==df_train['mileage'].max()]

In [33]:
# изменяем значение выброса на 99999
df_train.loc[df_train['mileage']==df_train['mileage'].max(), 'mileage'] = 99999

In [34]:
## mileage train AGAIN
numb_type_analisys(df_train, 'mileage', 
                   bins_step=(df_train.mileage.max() - df_train.mileage.min())//100)

In [35]:
## mileage test
numb_type_analisys(df_test, 'mileage', 
                   bins_step=(df_test.mileage.max() - df_test.mileage.min())//100)

In [36]:
df_test[df_test['mileage']==df_test['mileage'].max()]

Только одного авто в тестовом наборе имеется пробег 1000000 км и авто с пробегом более 500000 км нет. Т.к. в описании не указан, какой пробег, то исключаем данную строчку

In [37]:
df_test = df_test.loc[df_test['mileage']<1000000] 
# df_test.loc[df_test['mileage']<df_test['mileage'].max()]

In [38]:
## mileage test AGAIN
numb_type_analisys(df_test, 'mileage', 
                   bins_step=(df_test.mileage.max() - df_test.mileage.min())//100)

In [39]:
## looking for log hist of train mileage
col = 'mileage'
np.log(df_train[col]).hist(bins=np.arange(min(np.log(1+df_train[col])), 
                                       max(np.log(1+df_train[col])), 0.1),
                           align='left',
                           label='df_train '+col)
plt.legend()

# логарифмирование изменяет характер распределения и уменьшает масштаб числового признака

In [40]:
## looking for log hist of test mileage 
col = 'mileage'
np.log(df_test[col]).hist(bins=np.arange(min(np.log(1+df_test[col])), 
                                       max(np.log(1+df_test[col])), 0.1),
                           align='left',
                           label='df_test '+col)
plt.legend()

# логарифмирование изменяет характер распределения и уменьшает масштаб числового признака

### **modelDate**

In [41]:
## modelDate train
col='modelDate'
numb_type_analisys(df_train, col, bins_step=1)

In [42]:
df_train[df_train['modelDate']==df_train['modelDate'].min()]

In [43]:
# Смотрим, что на картинку предполагаемого выброса в train
image = PIL.Image.open('../input/sf-dst-car-price-prediction-part2/img/img/'+
                       str(df_train[df_train['modelDate']==df_train['modelDate'].min()].sell_id.values[0])+
                       '.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

# Что ж, похоже что авто, действительно 1975 года, оставляем =)

In [44]:
## modelDate test
numb_type_analisys(df_test, col, bins_step=1)

In [45]:
df_test[df_test['modelDate']==df_test['modelDate'].min()]

In [46]:
# Смотрим, что на картинку предполагаемого выброса в test
image = PIL.Image.open('../input/sf-dst-car-price-prediction-part2/img/img/'+
                       str(df_test[df_test['modelDate']==df_test['modelDate'].min()].sell_id.values[0])+
                       '.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

# И этот авто похож на авто 1971 года, оставляем =)

In [47]:
## looking log hist of train modelDate
col = 'modelDate'
np.log(df_train[col]).hist(bins=np.arange(min(np.log(df_train[col])), 
                                       max(np.log(df_train[col])), 0.0005),
                           align='left',
                           label=col)
plt.legend()
# логарифмирование сильно не изменяет характер распределения, только уменьшает масштаб

In [48]:
## looking log hist of test modelDate
col = 'modelDate'
np.log(df_test[col]).hist(bins=np.arange(min(np.log(df_test[col])), 
                                       max(np.log(df_test[col])), 0.0005),
                           align='left',
                           label=col)
plt.legend()
# логарифмирование сильно не изменяет характер распределения, только уменьшает масштаб


### **numberOfDoors**


In [49]:
## numberOfDoors df_train
col='numberOfDoors'
numb_type_analisys(df_train, col, bins_step=0.5)

In [50]:
## numberOfDoors df_test
col='numberOfDoors'
numb_type_analisys(df_test, col, bins_step=0.5)

### Распределения в целом схожи, двухдверные авто бывают, считаем, что выбросов нет.

## **productionDate**

In [51]:
## productionDate df_train
col='productionDate'
numb_type_analisys(df_train, col, bins_step=1)

In [52]:
## productionDate df_test
col='productionDate'
numb_type_analisys(df_test, col, bins_step=1)

### modelDate & productionDate скорее всего будут сильно скоррелированы и будем избавляться от одного из признаков

## **Владение**

In [53]:
## Владение df_train исключаем 0 из анализа, т.к. сделано искуственно из NaN и слишком много
col='Владение'
numb_type_analisys(df_train[df_train['Владение']>0], col, bins_step=1)

# Похоже, что выбросов нет, т.к. возраст авто и время владения близки
#см. df_train[df_train['Владение']>300]

In [54]:
## Владение df_test исключаем 0 из анализа, т.к. сделано искуственно из NaN и слишком много
col='Владение'
numb_type_analisys(df_test[df_test['Владение']>0], col, bins_step=1)

# Похоже, что выбросов нет, т.к. возраст авто и время владения близки
#см. df_test[df_test['Владение']>300]

## **ch_engineDisplacement**

In [55]:
## ch_engineDisplacement df_train
col='ch_engineDisplacement'
numb_type_analisys(df_train, col, bins_step=0.1)

# результат ожидаем, больше всего моторов с двигателем 2 и 3 литра

In [56]:
## ch_engineDisplacement df_test
col='ch_engineDisplacement'
numb_type_analisys(df_test, col, bins_step=0.1)

# результат ожидаем, больше всего моторов с двигателем 2 и 3 литра

In [57]:
## looking log hist of train ch_engineDisplacement
col = 'ch_engineDisplacement'
np.log(df_train[col]).hist(bins=np.arange(min(np.log(1+df_train[col])), 
                                          max(np.log(1+df_train[col])), 0.05),
                           align='left',
                           label=col)
plt.legend()
# логарифмирование делает распределение более симметричным и уменьшает масштаб

In [58]:
## looking log hist of test ch_engineDisplacement
col = 'ch_engineDisplacement'
np.log(df_test[col]).hist(bins=np.arange(min(np.log(1+df_test[col])), 
                                         max(np.log(1+df_test[col])), 0.05),
                           align='left',
                           label=col)
plt.legend()

# логарифмирование делает распределение более симметричным и уменьшает масштаб

## **ch_enginePower**

In [59]:
## ch_enginePower df_train
col='ch_enginePower'
numb_type_analisys(df_train, col, bins_step=10)

In [60]:
## ch_enginePower df_test
col='ch_enginePower'
numb_type_analisys(df_test, col, bins_step=10)

In [61]:
## looking log hist of train ch_engineDisplacement
col = 'ch_enginePower'
np.log(df_train[col]).hist(bins=np.arange(min(np.log(1+df_train[col])), 
                                          max(np.log(1+df_train[col])), 0.05),
                           align='left',
                           label=col)
plt.legend()
# логарифмирование делает распределение более симмтричным и уменьшает масштаб

In [62]:
## looking log hist of test ch_engineDisplacement
col = 'ch_enginePower'
np.log(df_test[col]).hist(bins=np.arange(min(np.log(1+df_test[col])), 
                                         max(np.log(1+df_test[col])), 0.05),
                           align='left',
                           label=col)
plt.legend()
# логарифмирование делает распределение более симмтричным и уменьшает масштаб

In [63]:
# Смотрим еще раз, как изменяются графики после логарифмирования

numb_cols = [x for x in df_train.dtypes[df_train.dtypes!=object].index 
             if x not in ['sell_id', 'price', 'numberOfDoors']]

ind_j = 0

def plot_col(df, col, ind):
    axes[ind].hist(df[col], 
                   bins = 25,
                   align='left')
    axes[ind].set_title(col)
    
    axes[ind+1].hist(np.log(1+df_train[col]),
                          bins = 25,
                          align='left')
    axes[ind+1].set_title('log ' + col)
    
fig, axes = plt.subplots(1, 4, figsize=(16, 2))

for col in numb_cols:
    if ind_j < 3:
        plot_col(df_train, col, ind_j)
        ind_j += 2
    else:
        plt.show()
        ind_j = 0
        fig, axes = plt.subplots(1, 4, figsize=(16, 2))
        plot_col(df_train, col, ind_j)
        ind_j += 2

In [64]:
df_train.corr()

1. **engineDisplacement** и **enginePower** имеют сильную корреляцию, одновременное их использование не рекомендуется. В связи с этим и с тем, что enginePower имеет более сильную корреляцию с price (target-переменная), чем engineDisplacement: enginePower можно переделать в числовой признак (+ в тесте есть значения отсуствующие в train), а engineDisplacement - оставить в категориальном признаке.
2. Аналогично для **modelDate** и **productionDate**, пробуем productionDate переделать в числовой признак, а modelDate - исключим.
3. **mileage** имеет сильную обратную корреляцию с modelDate и productionDate. Для начала попробуем посмотреть на рассчитанное качество.
2. target переменная price:
- имеет прямую сильную корреляцию с enginePower, engineDisplacement;
- имеет обратную сильную корреляцию с mileage;
- имеет прямую сильную корреляцию с modelDate, productionDate;

In [65]:
# сморим матрицу корреляции для np.log(df_train), выводы примерно те же, что и без np.log
df_train_log = np.log(1+df_train[df_train.dtypes[df_train.dtypes!=object].index].copy())
df_train_log.corr()

## Анализ номинативных переменных

In [66]:
label_cols = list(df_train.dtypes[df_train.dtypes==object].index)
label_cols

In [67]:
## 4. Анализ номинативных переменных
def get_boxplot(column, data):
    fig, ax = plt.subplots(figsize=(12, 5))
    sns.boxplot(x=column, y='price',
                data=data,  # данных в столбцах не много
                ax=ax)
    plt.xticks(rotation=90)
    ax.set_title('Boxplot for ' + column)
    plt.show()

# data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:10])]
# когда в столбце много значений и выделяем 10 наиболее встречающихся 

In [68]:
column = 'bodyType'
data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:10])]
get_boxplot(column, data)

In [69]:
column = 'brand'
data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:10])]
get_boxplot(column, data)

In [70]:
column = 'color'
data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:10])]
get_boxplot(column, data)

In [71]:
column = 'engineDisplacement'
data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:15])]
get_boxplot(column, data)

In [72]:
column = 'fuelType'
data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:15])]
get_boxplot(column, data)

In [73]:
column = 'model_info'
data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:15])]
get_boxplot(column, data)

In [74]:
column = 'vehicleConfiguration'
data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:15])]
get_boxplot(column, data)

In [75]:
column = 'vehicleTransmission'
data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:15])]
get_boxplot(column, data)

In [76]:
column = 'Владельцы'
data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:15])]
get_boxplot(column, data)

In [77]:
column = 'ПТС'
data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:15])]
get_boxplot(column, data)

In [78]:
column = 'Привод'
data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:15])]
get_boxplot(column, data)

In [79]:
column = 'Руль'
data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:15])]
get_boxplot(column, data)

In [80]:
column = 'ch_name'
data=df_train.loc[df_train.loc[:, column].isin(df_train.loc[:, column].value_counts().index[:15])]
get_boxplot(column, data)

По графикам похоже, что все параметры могут влиять на оценку стоимости авто. Графики являются вспомогательным инструментом. Проверим, есть ли статистическая разница в распределении стоимости авто по номинативным признакам, с помощью теста Стьюдента. Проверим нулевую гипотезу о том, что распределения стоимости авто по различным параметрам неразличимы

In [81]:
def get_stat_dif(column, alpha):
    """ aplpha - уровень значимости"""

    cols = df_train.loc[:, column].value_counts().index[:]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(df_train.loc[df_train.loc[:, column] == comb[0], 'price'],
                     df_train.loc[df_train.loc[:, column] == comb[1], 'price']).pvalue \
                     <= alpha/len(combinations_all):  # Учли поправку Бонферони
            print(
                'Найдены статистически значимые различия для колонки {} с уровнем значимости {}'.format(
                    column, alpha))
            break
        else:
            print(
                'НЕ НАЙДЕНЫ статистически значимые различия для колонки {} с уровнем значимости {}'.format(
                    column, alpha))
            break

In [82]:
list(combinations(df_train.loc[:, 'brand'].value_counts().index[:], 2))

In [83]:
for col in list(set(label_cols)-set(['description'])):
        get_stat_dif(col, 0.01)

In [84]:
for col in list(set(label_cols)-set(['description'])):
        get_stat_dif(col, 0.05)

In [85]:
for col in list(set(label_cols)-set(['description'])):
        get_stat_dif(col, 0.1)

### **Вывод**: на основании проведенного анализа при обучении модели можно исключить признак 'Руль' и возможно изменить преобразование признака 'name' до нахождения данных из признака 'name', которые дадут статистически значимые различия при проверке нулевой гипотезы. Иначе, также исключить данный признак.